In [23]:
import numpy as np
import pytorch_forecasting
import torch
from sklearn.preprocessing import MinMaxScaler
import random
import cv2
from scipy import stats
import pandas as pd
from dataclasses import make_dataclass
import pdb
from processdata import load_data
from processdata import TimeSeriesDataset
import models
import scipy.io
import pickle

num_sensors = 3 
lags = 50
load_X = np.load('pdf_data.npy')
n = load_X.shape[0]
m = load_X.shape[1]
sensor_locations = np.random.choice(m, size=num_sensors, replace=False)

We now select indices to divide the data into training, validation, and test sets.

In [25]:
train_indices = np.random.choice(n - lags, size=2500, replace=False)
mask = np.ones(n - lags)
mask[train_indices] = 0
valid_test_indices = np.arange(0, n - lags)[np.where(mask!=0)[0]]
valid_indices = valid_test_indices[::2]
test_indices = valid_test_indices[1::2]

In [26]:
sc = MinMaxScaler()
sc = sc.fit(load_X[train_indices])
transformed_X = sc.transform(load_X)

### Generate input sequences to a SHRED model
all_data_in = np.zeros((n - lags, lags, num_sensors))
for i in range(len(all_data_in)):
    all_data_in[i] = transformed_X[i:i+lags, sensor_locations]

### Generate training validation and test datasets both for reconstruction of states and forecasting sensors
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data_in = torch.tensor(all_data_in[train_indices], dtype=torch.float32).to(device)
valid_data_in = torch.tensor(all_data_in[valid_indices], dtype=torch.float32).to(device)
test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)

### -1 to have output be at the same time as final sensor measurements
train_data_out = torch.tensor(transformed_X[train_indices + lags - 1], dtype=torch.float32).to(device)
valid_data_out = torch.tensor(transformed_X[valid_indices + lags - 1], dtype=torch.float32).to(device)
test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

train_dataset = TimeSeriesDataset(train_data_in, train_data_out)
valid_dataset = TimeSeriesDataset(valid_data_in, valid_data_out)
test_dataset = TimeSeriesDataset(test_data_in, test_data_out)

In [28]:
shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)
validation_errors = models.fit(shred, train_dataset, valid_dataset, batch_size=64, num_epochs=1000, lr=1e-3, verbose=True, patience=5)

Training epoch 1
Error tensor(0.3505)
Training epoch 20
Error tensor(0.3147)
Training epoch 40
Error tensor(0.2539)
Training epoch 60
Error tensor(0.1667)
Training epoch 80
Error tensor(0.1003)
Training epoch 100
Error tensor(0.0865)
Training epoch 120
Error tensor(0.0817)
Training epoch 140
Error tensor(0.1329)
Training epoch 160
Error tensor(0.0756)
Training epoch 180
Error tensor(0.0719)
Training epoch 200
Error tensor(0.0697)
Training epoch 220
Error tensor(0.0691)
Training epoch 240
Error tensor(0.0678)
Training epoch 260
Error tensor(0.0660)
Training epoch 280
Error tensor(0.0657)
Training epoch 300
Error tensor(0.0641)
Training epoch 320
Error tensor(0.0643)
Training epoch 340
Error tensor(0.0628)
Training epoch 360
Error tensor(0.0627)
Training epoch 380
Error tensor(0.0629)
Training epoch 400
Error tensor(0.0633)
Training epoch 420
Error tensor(0.1206)
Training epoch 440
Error tensor(0.0617)
Training epoch 460
Error tensor(0.0612)
Training epoch 480
Error tensor(0.0618)
Traini

KeyboardInterrupt: 

In [29]:
test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())
print(np.linalg.norm(test_recons - test_ground_truth) / np.linalg.norm(test_ground_truth))

0.019781288
